In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Cristóvão Beirão da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
import importlib
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from fnmatch import fnmatch
import scipy.stats as stats
import hist
import mplhep as hep
plt.style.use(hep.style.CMS)
from notebooks.notebook_helpers import *

from scripts.log_action import log_action_v2
from time import sleep
from scripts.read_current_v2 import DeviceMeasurements


os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
importlib.reload(run_script)

parser = run_script.getOptionParser()

# Get a controller to control the power supplies

In [ ]:
device_meas = DeviceMeasurements(
                                outdir = Path("../i2c_gui/"),
                                #outdir = Path('/run/media/daq/T7/'),
                                interval = 5,  # measure every 5 seconds
                                #baudrate = args.baudrate,
                                write_termination = '\n',
                                read_termination = '\r\n',
                                 )

device_meas.add_instrument("Power", "THURLBY THANDAR", "PL303QMD-P", "506013")
device_meas.add_instrument("WS Power", "THURLBY THANDAR", "PL303QMD-P", "521246")

device_meas.add_channel("Power", 1, "Analog", config = {
                                                        "Vset": 1.2 + 0.04,
                                                        "Ilimit": 0.5,
                                                        "IRange": "Low",  # Alternative "High"
                                                        }
)
device_meas.add_channel("Power", 2, "Digital", config = {
                                                        "Vset": 1.2 + 0.09,
                                                        "Ilimit": 0.4,
                                                        "IRange": "Low",  # Alternative "High"
                                                        }
)
device_meas.add_channel("WS Power", 1, "Analog", config = {
                                                        "Vset": 1.2 + 0.01,
                                                        "Ilimit": 0.03,
                                                        "IRange": "Low",  # Alternative "High"
                                                        }
)
device_meas.add_channel("WS Power", 2, "Digital", config = {
                                                        "Vset": 1.2 + 0.01,
                                                        "Ilimit": 0.1,
                                                        "IRange": "Low",  # Alternative "High"
                                                        }
)

device_meas.find_devices()

In [ ]:
device_meas.turn_on()

# Set defaults

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_01E2_52"]
# chip_names = ["ET2-W36-IP5-20", "ET2-W36-IP7-13", "ET2-W36-IP7-10"]
# old 
# chip_names = ["ET2-W36-IP7-12", "ET2-W36-IP5-15", "ET2-W36-IP5-14"]
# old [0x78, 0x61, 0x74]
# old top modified C type =  ET2-W36-IP5-15
# replaced with new type D = ET2-W36-IP7-10
chip_fignames = chip_names
chip_figtitles = ["ETROC2 01E2 #52 21C PostTID 200MRads"]

# 'The port name the USB-ISS module is connected to. Default: COM3'
port = "/dev/ttyACM0"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60]
# chip_addresses = [0x78, 0x74, 0x61]
ws_addresses = [None]

today = datetime.date.today().isoformat()
# today = "2023-09-14"


# i2c_gui.__no_connect__ = False  # Set to fake connecting to an ETROC2 device
# i2c_gui.__no_connect_type__ = "echo"  # for actually testing readback
# #i2c_gui.__no_connect_type__ = "check"  # default behaviour

hostname = "192.168.2.3"

polarity = "0x000f"

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# Make i2c_connection class object

In [ ]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,chip_names,[("1","1")])

# Config chips

### Key is (Disable Pixels, Auto Cal, Chip Peripherals, Basic Peri Reg Check, Pixel Check)

In [ ]:
log_action_v2(Path("../i2c_gui"), "Config", "", f"Initial chip configuration for chip {chip_names[0]}")

# 0 - 0 - (disable & auto_cal all pixels) - (auto_TH_CAL) - (disable default all pixels) - (set basic peripherals) - (peripheral reg check) -  (pixel ID check)
# i2c_conn.config_chips('00100111')
i2c_conn.config_chips('00010000')

## Visualize the learned Baselines (BL) and Noise Widths (NW)

Note that the NW represents the full width on either side of the BL

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(dpi=50, figsize=(5,5))
gs = fig.add_gridspec(1,1)

ax0 = fig.add_subplot(gs[0,0])
ax0.plot([1, 0], [1, 0])
plt.show()
plt.close()

In [ ]:
# from mpl_toolkits.axes_grid1 import make_axes_locatable
for chip_address, chip_figname, chip_figtitle in zip(chip_addresses,chip_fignames,chip_figtitles):
    BL_map_THCal,NW_map_THCal,BL_df = i2c_conn.get_auto_cal_maps(chip_address)
    fig = plt.figure(dpi=200, figsize=(20,10))
    gs = fig.add_gridspec(1,2)

    ax0 = fig.add_subplot(gs[0,0])
    ax0.set_title(f"{chip_figtitle}: BL (DAC LSB)", size=17, loc="right")
    img0 = ax0.imshow(BL_map_THCal, interpolation='none')
    ax0.set_aspect("equal")
    ax0.invert_xaxis()
    ax0.invert_yaxis()
    plt.xticks(range(16), range(16), rotation="vertical")
    plt.yticks(range(16), range(16))
    hep.cms.text(loc=0, ax=ax0, fontsize=17, text="Preliminary")
    divider = make_axes_locatable(ax0)
    cax = divider.append_axes('right', size="5%", pad=0.05)
    fig.colorbar(img0, cax=cax, orientation="vertical")

    ax1 = fig.add_subplot(gs[0,1])
    ax1.set_title(f"{chip_figtitle}: NW (DAC LSB)", size=17, loc="right")
    img1 = ax1.imshow(NW_map_THCal, interpolation='none')
    ax1.set_aspect("equal")
    ax1.invert_xaxis()
    ax1.invert_yaxis()
    plt.xticks(range(16), range(16), rotation="vertical")
    plt.yticks(range(16), range(16))
    hep.cms.text(loc=0, ax=ax1, fontsize=17, text="Preliminary")
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size="5%", pad=0.05)
    fig.colorbar(img1, cax=cax, orientation="vertical")

    for x in range(16):
        for y in range(16):
            ax0.text(x,y,f"{BL_map_THCal.T[x,y]:.0f}", c="white", size=10, rotation=45, fontweight="bold", ha="center", va="center")
            ax1.text(x,y,f"{NW_map_THCal.T[x,y]:.0f}", c="white", size=10, rotation=45, fontweight="bold", ha="center", va="center")
    plt.tight_layout()
    plt.savefig(fig_path+"/BL_NW_"+chip_figname+"_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    plt.show()

### Save BL and NW to be loaded later

In [ ]:
for chip_address, chip_name in zip(chip_addresses, chip_names):
    i2c_conn.save_auto_cal_BL_map(chip_address, chip_name, "")
    i2c_conn.save_auto_cal_NW_map(chip_address, chip_name, "")

In [ ]:
for chip_address, chip_name in zip(chip_addresses, chip_names):
    i2c_conn.load_auto_cal_BL_map(chip_address, chip_name, "")
    i2c_conn.load_auto_cal_NW_map(chip_address, chip_name, "")

## Save in git

In [ ]:
outdir = Path('../ETROC-Data')
outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
outdir.mkdir(exist_ok=True)
outfile = outdir / (chip_fignames[0]+"_BaselineAt_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
BL_df.to_csv(outfile, index=False)

In [ ]:
note = 'PostTID_200MRads_Plus21'+chip_names[0]
new_columns = {
    'note': f'{note}',
}

for col in new_columns:
    BL_df[col] = new_columns[col]

outdir = Path('../ETROC-History')
outfile = outdir / 'BaselineHistory.sqlite'

init_cmd = [
    'cd ' + str(outdir.resolve()),
    'git stash -u',
    'git pull',
]
end_cmd = [
    'cd ' + str(outdir.resolve()),
    'git add BaselineHistory.sqlite',
    'git commit -m "Added new history entry"',
    'git push',
    'git stash pop',
    'git stash clear',
]
init_cmd = [x + '\n' for x in init_cmd]
end_cmd  = [x + '\n' for x in end_cmd]

p = subprocess.Popen(
    '/bin/bash',
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    encoding="utf-8",
    )

for cmd in init_cmd:
    p.stdin.write(cmd + "\n")
p.stdin.close()
p.wait()

print(p.stdout.read())

with sqlite3.connect(outfile) as sqlconn:
    BL_df.to_sql('baselines', sqlconn, if_exists='append', index=False)

p = subprocess.Popen(
    '/bin/bash',
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    encoding="utf-8",
    )

for cmd in end_cmd:
    p.stdin.write(cmd + "\n")
p.stdin.close()
p.wait()

p.stdin.close()

print(p.stdout.read())


# Calibrate PLL

In [ ]:
log_action_v2(Path("../i2c_gui"), "Config", "", "Calibrating PLL")

for chip_address in chip_addresses[:]:
    i2c_conn.calibratePLL(chip_address, chip=None)

# Calibrate FC

In [ ]:
log_action_v2(Path("../i2c_gui"), "Config", "", "Calibrating FC")

for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)

# Define pixels of interest

In [ ]:
row_list = [0,  0, 15, 15]
col_list = [0, 15, 15,  0]
qinj_list = [30]

scan_list = list(zip(row_list, col_list))
print(scan_list)

# Run One Time DAQ to Set FPGA Firmware

In [ ]:
log_action_v2(Path("../i2c_gui"), "Config", "", "Run one time DAQ")

parser = run_script.getOptionParser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 20 -o CanBeRemoved_Board0_NoLinkCheck -v -w -s 0x0000 -p {polarity} -d 0x1800 -a 0x0011 --clear_fifo".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs_Board0_NoLinkCheck'))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj L1A BCR')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

# Scan DAC to find offset above double peak

In [ ]:
log_action_v2(Path("../i2c_gui"), "Config", "", "Performing DAC QInj scan")

scan_name = chip_fignames[0]+"_DACScan"
fpga_time = 3
scan_step = 2


parser = run_script.getOptionParser()

for QInj in qinj_list:
    for (col, row) in scan_list:
        print(f"Pixel: {col},{row}; QInj: {QInj}")

        pixel_connected_chip = i2c_conn.get_pixel_chip(chip_address, row, col)
        # Enable charge injection
        i2c_conn.pixel_decoded_register_write("disDataReadout", "0", pixel_connected_chip)
        i2c_conn.pixel_decoded_register_write("QInjEn", "1", pixel_connected_chip)
        i2c_conn.pixel_decoded_register_write("disTrigPath", "0", pixel_connected_chip)
        # Bypass Cal Threshold
        i2c_conn.pixel_decoded_register_write("Bypass_THCal", "1", pixel_connected_chip)

        # Modifying charge injected
        i2c_conn.pixel_decoded_register_write("QSel", format(QInj, '05b'), pixel_connected_chip)

        threshold_name = scan_name+f'_Pixel_C{col}_R{row}_QInj_{QInj}'

        # -s 0x0000 -a 0x0011 -p 0x000f -d 0x0800
        (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -o {threshold_name} -v -w --reset_till_trigger_linked -s 0x0000 -p 0x000f -d 0x1800 -c 0x0001 --fpga_data_time_limit 3 --fpga_data_QInj -a 0x0011 --check_trigger_link_at_end --nodaq --clear_fifo".split())
        IPC_queue = multiprocessing.Queue()
        process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_link'))
        process.start()
        process.join()

        thresholds = np.arange(BL_map_THCal[row][col]-20, BL_map_THCal[row][col]+20, scan_step)
        thresholds = np.arange(BL_map_THCal[row][col]-2, BL_map_THCal[row][col]+2, scan_step)

        for DAC in tqdm(thresholds, desc=f'DAC Loop for Pixel ({col},{row}) & Charge {QInj} fC', leave=False):
            # Set the DAC v, Qinj {Qinj}fCalue to the value being scanned
            i2c_conn.pixel_decoded_register_write("DAC", format(int(DAC), '010b'), pixel_connected_chip)
            (options, args) = parser.parse_args(args=f"--useIPC --hostname {hostname} -o {threshold_name} -v --reset_till_trigger_linked -s 0x0000 -p 0x000f -d 0x1800 -c 0x0001 --fpga_data_time_limit {int(fpga_time)} --fpga_data_QInj --check_trigger_link_at_end -a 0x0011 --nodaq --DAC_Val {int(DAC)}".split()) # -s 0x0000 -a 0x0011 -p 0x000f -d 0x0800
            IPC_queue = multiprocessing.Queue()
            process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'process_outputs/main_process_Pixel_C{col}_R{row}_{QInj}_{int(DAC)}'))
            process.start()

            process.join()
            
        # Disable charge injection
        i2c_conn.pixel_decoded_register_write("QInjEn", "0", pixel_connected_chip)
        i2c_conn.pixel_decoded_register_write("disDataReadout", "1", pixel_connected_chip)
        i2c_conn.pixel_decoded_register_write("disTrigPath", "1", pixel_connected_chip)
        i2c_conn.pixel_decoded_register_write("DAC", format(0x3ff, '010b'), pixel_connected_chip)

### Enable pixels of Interest

In [ ]:
i2c_conn.enable_select_pixels_in_chips(scan_list)

In [ ]:
offset = 0x18
for chip_address in chip_addresses[:]:
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row, col in scan_list:
        print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col})")
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)    
        i2c_conn.pixel_decoded_register_write("QSel", format(0x14, '05b'), chip)
        i2c_conn.pixel_decoded_register_write("TH_offset", format(offset, '06b'), chip)
    del chip, row_indexer_handle, column_indexer_handle

### Disable Pixels of Interest

In [ ]:
for chip_address in chip_addresses[:]:
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")
    for row,col in scan_list:
        i2c_conn.disable_pixel(row=row, col=col, verbose=True, chip_address=chip_address, chip=chip, row_indexer_handle=row_indexer_handle, column_indexer_handle=column_indexer_handle)
    del chip, row_indexer_handle, column_indexer_handle

In [ ]:
device_meas.turn_off()

In [ ]:
device_meas.release_devices()

In [ ]:
#Examples - Channel 1 on right side, channel 2 on left side
# Channel 1 is always analog
# channel 2 is always digital
# There are 2 instruments, "Power" and "WS Power"

# Set channel 2 of Power to 1.1 V.
device_meas.set_channel_voltage("Power", 2, 1.1) 

In [ ]:
voltages = [1.1, 1.2, 1.3]

for voltage in voltages:
    device_meas.set_channel_voltage("Power", 1, voltage)

    # take data
    meas = device_meas.do_measurement()